<a href="https://colab.research.google.com/github/sabrinabenb/Graph-UNet/blob/main/Graph_Unet%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this work we combine several innovative elements to
boost its effectiveness. It utilizes data enrichment methods to compensate
for incomplete data, refined optimization approaches to improve learning efficiency,
and targeted regularization strategies to address common GNN limitations.
These elements work in concert to overcome fundamental challenges
in graph representation learning while maintaining scalability across various
graph sizes and types

In [ ]:
%%capture
import os
import torch
os.environ['TORCH'] = torch.__version__
os.environ['PYTHONWARNINGS'] = "ignore"
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
from typing import Callable, List, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch_sparse import spspmm

from torch_geometric.typing import OptTensor, PairTensor
from torch_geometric.utils import (
    add_self_loops,
    remove_self_loops,
    sort_edge_index,
)
from torch_geometric.utils.repeat import repeat
%matplotlib inline


In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='citeseer', transform=NormalizeFeatures())

print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]
g = dataset[0] # Get the first graph object.
print(data)


Processing...


Dataset: citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


Done!


#New version of Graph UNets

DeepGUNet model combining GCNConv, SAGPooling, and LayerNorm

In [ ]:
from typing import Callable, List, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch_sparse import spspmm

from torch_geometric.nn import GCNConv, SAGPooling,TopKPooling,GATConv,SSGConv, LayerNorm # Import LayerNorm
from torch_geometric.typing import OptTensor, PairTensor
from torch_geometric.utils import (
    add_self_loops,
    remove_self_loops,
    sort_edge_index,
)
from torch_geometric.utils.repeat import repeat

class DeepGUNet(torch.nn.Module):
    def __init__(
        self,
        in_channels: int,
        hidden_channels: int,
        out_channels: int,
        depth= int,
        pool_ratios: Union[float, List[float]] = 0.5,
        sum_res: bool = True,
        act: Callable = F.relu,
        dropout: float = 0.5 # Added dropout rate parameter
    ) -> None:
        super().__init__()

        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = repeat(pool_ratios, depth)
        self.act = act
        self.sum_res = sum_res
        self.dropout = dropout # Store dropout rate

        channels = hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(in_channels, channels,improved=True))
        self.norms = torch.nn.ModuleList()
        self.norms.append(LayerNorm(channels))
        # Added dropout layers for downsampling
        self.down_dropouts = torch.nn.ModuleList()
        self.down_dropouts.append(torch.nn.Dropout(self.dropout))

        for i in range(depth):
            self.pools.append(SAGPooling(channels, self.pool_ratios[i]))
            self.down_convs.append(GCNConv(channels, channels,improved=True))
            self.norms.append(LayerNorm(channels))
            self.down_dropouts.append(torch.nn.Dropout(self.dropout)) # Added dropout layers


        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        # Added dropout layers for upsampling
        self.up_dropouts = torch.nn.ModuleList()
        for i in range(depth): # Need dropout for each up_conv including the last one
             self.up_convs.append(GCNConv(in_channels, channels,improved=True) if i < depth -1 else SSGConv(in_channels, out_channels,alpha=0.3))
             self.up_dropouts.append(torch.nn.Dropout(self.dropout))


        self.reset_parameters()

    def reset_parameters(self):
        for conv in self.down_convs:
            conv.reset_parameters()
        for pool in self.pools:
            pool.reset_parameters()
        for conv in self.up_convs:
            conv.reset_parameters()
        for norm in self.norms:
            norm.reset_parameters()
        # Removed reset_parameters() calls for dropout layers


    def forward(self, x: Tensor, edge_index: Tensor,
                batch: OptTensor = None) -> Tensor:
        """"""
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        edge_weight = x.new_ones(edge_index.size(1))

        x = self.down_convs[0](x, edge_index, edge_weight)
        x = self.norms[0](x)
        x = self.act(x)
        x = self.down_dropouts[0](x) # Applied dropout

        xs = [x]
        edge_indices = [edge_index]
        edge_weights = [edge_weight]
        perms = []

        for i in range(1, self.depth + 1):

            x, edge_index, edge_weight, batch, perm, _ = self.pools[i - 1](
                x, edge_index, edge_weight, batch)

            x = self.down_convs[i](x, edge_index, edge_weight)
            x = self.norms[i](x)
            x = self.act(x)
            x = self.down_dropouts[i](x) # Applied dropout


            if i < self.depth:
                xs += [x]
                edge_indices += [edge_index]
                edge_weights += [edge_weight]
            perms += [perm]

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            edge_weight = edge_weights[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)

            x = self.up_convs[i](x, edge_index, edge_weight)

            # Applied dropout after activation in upsampling
            x = self.act(x) if i < self.depth - 1 else x
            x = self.up_dropouts[i](x) if i < self.depth -1 else x # Only apply dropout before the final output layer


        return x

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F

from torch_geometric.utils import dropout_edge

#depth=3

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=3, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2083, Val: 0.1960, Test: 0.2230
Epoch: 002, Train: 0.3167, Val: 0.1960, Test: 0.2230
Epoch: 003, Train: 0.3583, Val: 0.2480, Test: 0.2340
Epoch: 004, Train: 0.4417, Val: 0.2820, Test: 0.2980
Epoch: 005, Train: 0.5500, Val: 0.3340, Test: 0.3470
Epoch: 006, Train: 0.6667, Val: 0.3860, Test: 0.3970
Epoch: 007, Train: 0.7083, Val: 0.4000, Test: 0.4050
Epoch: 008, Train: 0.6750, Val: 0.4040, Test: 0.3930
Epoch: 009, Train: 0.7167, Val: 0.4100, Test: 0.4260
Epoch: 010, Train: 0.7667, Val: 0.4580, Test: 0.4700
Epoch: 011, Train: 0.8167, Val: 0.4780, Test: 0.4830
Epoch: 012, Train: 0.8083, Val: 0.5120, Test: 0.5220
Epoch: 013, Train: 0.8750, Val: 0.5760, Test: 0.5600
Epoch: 014, Train: 0.9083, Val: 0.6220, Test: 0.6180
Epoch: 015, Train: 0.9083, Val: 0.6480, Test: 0.6600
Epoch: 016, Train: 0.8917, Val: 0.6680, Test: 0.6770
Epoch: 017, Train: 0.9083, Val: 0.6740, Test: 0.6890
Epoch: 018, Train: 0.9167, Val: 0.6740, Test: 0.6890
Epoch: 019, Train: 0.9333, Val: 0.6740, Test: 

#Depth 5

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=5, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2500, Val: 0.1580, Test: 0.2210
Epoch: 002, Train: 0.2250, Val: 0.1580, Test: 0.2210
Epoch: 003, Train: 0.3667, Val: 0.1800, Test: 0.2070
Epoch: 004, Train: 0.5000, Val: 0.3440, Test: 0.3370
Epoch: 005, Train: 0.5417, Val: 0.4020, Test: 0.3660
Epoch: 006, Train: 0.5000, Val: 0.4020, Test: 0.3660
Epoch: 007, Train: 0.5333, Val: 0.4520, Test: 0.4340
Epoch: 008, Train: 0.5833, Val: 0.4600, Test: 0.4730
Epoch: 009, Train: 0.6083, Val: 0.4600, Test: 0.4730
Epoch: 010, Train: 0.6583, Val: 0.4600, Test: 0.4730
Epoch: 011, Train: 0.8083, Val: 0.5480, Test: 0.5650
Epoch: 012, Train: 0.8500, Val: 0.5620, Test: 0.5880
Epoch: 013, Train: 0.8667, Val: 0.5620, Test: 0.5880
Epoch: 014, Train: 0.8667, Val: 0.5620, Test: 0.5880
Epoch: 015, Train: 0.8917, Val: 0.5660, Test: 0.5980
Epoch: 016, Train: 0.9083, Val: 0.5900, Test: 0.6180
Epoch: 017, Train: 0.9333, Val: 0.6220, Test: 0.6410
Epoch: 018, Train: 0.9417, Val: 0.6440, Test: 0.6600
Epoch: 019, Train: 0.9500, Val: 0.6560, Test: 

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=4, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.1750, Val: 0.0540, Test: 0.0780
Epoch: 002, Train: 0.2500, Val: 0.0780, Test: 0.0920
Epoch: 003, Train: 0.2083, Val: 0.1100, Test: 0.1210
Epoch: 004, Train: 0.3750, Val: 0.3220, Test: 0.3230
Epoch: 005, Train: 0.4500, Val: 0.3460, Test: 0.3710
Epoch: 006, Train: 0.6083, Val: 0.3900, Test: 0.4450
Epoch: 007, Train: 0.7167, Val: 0.4460, Test: 0.4700
Epoch: 008, Train: 0.7333, Val: 0.4460, Test: 0.4700
Epoch: 009, Train: 0.7333, Val: 0.4460, Test: 0.4700
Epoch: 010, Train: 0.7167, Val: 0.4460, Test: 0.4700
Epoch: 011, Train: 0.8000, Val: 0.4760, Test: 0.4750
Epoch: 012, Train: 0.8083, Val: 0.4980, Test: 0.4900
Epoch: 013, Train: 0.8083, Val: 0.5680, Test: 0.5500
Epoch: 014, Train: 0.8333, Val: 0.6220, Test: 0.5790
Epoch: 015, Train: 0.8250, Val: 0.6220, Test: 0.5790
Epoch: 016, Train: 0.8417, Val: 0.6220, Test: 0.5790
Epoch: 017, Train: 0.8583, Val: 0.6520, Test: 0.6480
Epoch: 018, Train: 0.8833, Val: 0.6680, Test: 0.6770
Epoch: 019, Train: 0.9000, Val: 0.6740, Test: 

#Depth 4

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=13, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2000, Val: 0.1020, Test: 0.1020
Epoch: 002, Train: 0.2417, Val: 0.2180, Test: 0.2080
Epoch: 003, Train: 0.2333, Val: 0.2180, Test: 0.2080
Epoch: 004, Train: 0.3583, Val: 0.3560, Test: 0.3270
Epoch: 005, Train: 0.4000, Val: 0.3560, Test: 0.3270
Epoch: 006, Train: 0.3917, Val: 0.3560, Test: 0.3270
Epoch: 007, Train: 0.5667, Val: 0.3600, Test: 0.3460
Epoch: 008, Train: 0.4417, Val: 0.3600, Test: 0.3460
Epoch: 009, Train: 0.4583, Val: 0.3600, Test: 0.3460
Epoch: 010, Train: 0.5750, Val: 0.3600, Test: 0.3460
Epoch: 011, Train: 0.6583, Val: 0.3600, Test: 0.3460
Epoch: 012, Train: 0.6917, Val: 0.3600, Test: 0.3460
Epoch: 013, Train: 0.7000, Val: 0.3860, Test: 0.3720
Epoch: 014, Train: 0.7583, Val: 0.4320, Test: 0.4210
Epoch: 015, Train: 0.8167, Val: 0.5180, Test: 0.4920
Epoch: 016, Train: 0.8750, Val: 0.5680, Test: 0.5420
Epoch: 017, Train: 0.8917, Val: 0.5840, Test: 0.5730
Epoch: 018, Train: 0.9000, Val: 0.5840, Test: 0.5730
Epoch: 019, Train: 0.9167, Val: 0.5880, Test: 

#Depth 20

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=20, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.1917, Val: 0.2120, Test: 0.1870
Epoch: 002, Train: 0.1833, Val: 0.2120, Test: 0.1870
Epoch: 003, Train: 0.1750, Val: 0.2120, Test: 0.1870
Epoch: 004, Train: 0.2417, Val: 0.2120, Test: 0.1870
Epoch: 005, Train: 0.4000, Val: 0.2820, Test: 0.3250
Epoch: 006, Train: 0.4000, Val: 0.3320, Test: 0.3450
Epoch: 007, Train: 0.5250, Val: 0.3480, Test: 0.3430
Epoch: 008, Train: 0.4833, Val: 0.3480, Test: 0.3430
Epoch: 009, Train: 0.4750, Val: 0.3480, Test: 0.3430
Epoch: 010, Train: 0.4667, Val: 0.3480, Test: 0.3430
Epoch: 011, Train: 0.5750, Val: 0.3480, Test: 0.3430
Epoch: 012, Train: 0.6167, Val: 0.4180, Test: 0.4070
Epoch: 013, Train: 0.7083, Val: 0.4940, Test: 0.4940
Epoch: 014, Train: 0.8167, Val: 0.5720, Test: 0.5530
Epoch: 015, Train: 0.8500, Val: 0.5840, Test: 0.5560
Epoch: 016, Train: 0.8667, Val: 0.6120, Test: 0.5920
Epoch: 017, Train: 0.9167, Val: 0.6180, Test: 0.6170
Epoch: 018, Train: 0.9167, Val: 0.6180, Test: 0.6170
Epoch: 019, Train: 0.9250, Val: 0.6180, Test: 

#Depth 50

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=50, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2000, Val: 0.1220, Test: 0.1230
Epoch: 002, Train: 0.2917, Val: 0.1640, Test: 0.1660
Epoch: 003, Train: 0.3167, Val: 0.2280, Test: 0.2440
Epoch: 004, Train: 0.3917, Val: 0.3160, Test: 0.3030
Epoch: 005, Train: 0.4250, Val: 0.3220, Test: 0.3020
Epoch: 006, Train: 0.4500, Val: 0.3340, Test: 0.3220
Epoch: 007, Train: 0.5333, Val: 0.3920, Test: 0.3880
Epoch: 008, Train: 0.5667, Val: 0.4340, Test: 0.4150
Epoch: 009, Train: 0.5833, Val: 0.4420, Test: 0.4020
Epoch: 010, Train: 0.6750, Val: 0.4420, Test: 0.4020
Epoch: 011, Train: 0.7667, Val: 0.4420, Test: 0.4020
Epoch: 012, Train: 0.8083, Val: 0.4820, Test: 0.4730
Epoch: 013, Train: 0.8167, Val: 0.4820, Test: 0.4730
Epoch: 014, Train: 0.8583, Val: 0.4820, Test: 0.4730
Epoch: 015, Train: 0.8667, Val: 0.5100, Test: 0.5150
Epoch: 016, Train: 0.9000, Val: 0.5280, Test: 0.5540
Epoch: 017, Train: 0.9083, Val: 0.5720, Test: 0.5800
Epoch: 018, Train: 0.9167, Val: 0.6220, Test: 0.6120
Epoch: 019, Train: 0.8000, Val: 0.6220, Test: 

#Depth 100

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=100, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2167, Val: 0.1860, Test: 0.2300
Epoch: 002, Train: 0.2500, Val: 0.2820, Test: 0.2510
Epoch: 003, Train: 0.3583, Val: 0.2840, Test: 0.2440
Epoch: 004, Train: 0.4417, Val: 0.2840, Test: 0.2440
Epoch: 005, Train: 0.4250, Val: 0.2840, Test: 0.2440
Epoch: 006, Train: 0.5333, Val: 0.3240, Test: 0.3310
Epoch: 007, Train: 0.5833, Val: 0.3880, Test: 0.4030
Epoch: 008, Train: 0.5500, Val: 0.3920, Test: 0.4130
Epoch: 009, Train: 0.6000, Val: 0.4100, Test: 0.4280
Epoch: 010, Train: 0.6333, Val: 0.4120, Test: 0.4400
Epoch: 011, Train: 0.7250, Val: 0.4640, Test: 0.4780
Epoch: 012, Train: 0.8417, Val: 0.5340, Test: 0.5570
Epoch: 013, Train: 0.9000, Val: 0.5660, Test: 0.5730
Epoch: 014, Train: 0.9250, Val: 0.5800, Test: 0.5830
Epoch: 015, Train: 0.9250, Val: 0.5800, Test: 0.5830
Epoch: 016, Train: 0.9333, Val: 0.5820, Test: 0.5690
Epoch: 017, Train: 0.9250, Val: 0.5940, Test: 0.5740
Epoch: 018, Train: 0.9250, Val: 0.6040, Test: 0.6120
Epoch: 019, Train: 0.9250, Val: 0.6300, Test: 

#Depth 500

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=500, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.2250, Val: 0.1400, Test: 0.1580
Epoch: 002, Train: 0.3250, Val: 0.3020, Test: 0.2910
Epoch: 003, Train: 0.3500, Val: 0.3020, Test: 0.2910
Epoch: 004, Train: 0.3833, Val: 0.3120, Test: 0.2800
Epoch: 005, Train: 0.5333, Val: 0.3760, Test: 0.3760
Epoch: 006, Train: 0.5417, Val: 0.3760, Test: 0.3760
Epoch: 007, Train: 0.6250, Val: 0.4000, Test: 0.4180
Epoch: 008, Train: 0.7417, Val: 0.4620, Test: 0.4830
Epoch: 009, Train: 0.8667, Val: 0.5280, Test: 0.5540
Epoch: 010, Train: 0.8917, Val: 0.5840, Test: 0.5920
Epoch: 011, Train: 0.9167, Val: 0.5840, Test: 0.5920
Epoch: 012, Train: 0.9083, Val: 0.5840, Test: 0.5920
Epoch: 013, Train: 0.9167, Val: 0.5840, Test: 0.5920
Epoch: 014, Train: 0.9250, Val: 0.5840, Test: 0.5920
Epoch: 015, Train: 0.9417, Val: 0.5840, Test: 0.5920
Epoch: 016, Train: 0.9583, Val: 0.5840, Test: 0.5920
Epoch: 017, Train: 0.9500, Val: 0.5920, Test: 0.5910
Epoch: 018, Train: 0.9417, Val: 0.6340, Test: 0.6190
Epoch: 019, Train: 0.9417, Val: 0.6400, Test: 

KeyboardInterrupt: 

#Depth 1000

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        pool_ratios = [2000 / data.num_nodes, 0.5]
        self.unet = DeepGUNet(dataset.num_features, 32, dataset.num_classes,
                              depth=1000, pool_ratios=pool_ratios)

    def forward(self):
        edge_index, _ = dropout_edge(data.edge_index, p=0.2,
                                     force_undirected=True,
                                     training=self.training)
        x = F.dropout(data.x, p=0.92, training=self.training)

        x = self.unet(x, edge_index)
        return F.log_softmax(x, dim=1)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)


def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


@torch.no_grad()
def test():
    model.eval()
    out, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = out[mask].argmax(1)
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


best_val_acc = test_acc = 0
for epoch in range(1, 201):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.1750, Val: 0.2160, Test: 0.1920
Epoch: 002, Train: 0.3083, Val: 0.2700, Test: 0.2490
Epoch: 003, Train: 0.2333, Val: 0.2700, Test: 0.2490
Epoch: 004, Train: 0.3250, Val: 0.2760, Test: 0.2630
Epoch: 005, Train: 0.3333, Val: 0.2960, Test: 0.2800
Epoch: 006, Train: 0.5083, Val: 0.3820, Test: 0.3610
Epoch: 007, Train: 0.5250, Val: 0.3820, Test: 0.3610
Epoch: 008, Train: 0.5250, Val: 0.3820, Test: 0.3610
Epoch: 009, Train: 0.6083, Val: 0.3940, Test: 0.3930
Epoch: 010, Train: 0.6583, Val: 0.4680, Test: 0.4540
Epoch: 011, Train: 0.7333, Val: 0.4980, Test: 0.4770
Epoch: 012, Train: 0.7917, Val: 0.5080, Test: 0.4980
Epoch: 013, Train: 0.8667, Val: 0.5700, Test: 0.5720
Epoch: 014, Train: 0.9000, Val: 0.6320, Test: 0.6350
Epoch: 015, Train: 0.9000, Val: 0.6400, Test: 0.6380
Epoch: 016, Train: 0.9083, Val: 0.6600, Test: 0.6480
Epoch: 017, Train: 0.9167, Val: 0.6600, Test: 0.6480
Epoch: 018, Train: 0.9250, Val: 0.6600, Test: 0.6480
Epoch: 019, Train: 0.9250, Val: 0.6600, Test: 